In [14]:
# standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os # need for file enumeration
import pickle # to save model

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping

# For reproducibility
np.random.seed(42)

In [3]:
# get our file listings for train and test
train_files = os.listdir('images/train/')
test_files = os.listdir('images/test/')

In [4]:
# convert filenames into dataframes
train_df = pd.DataFrame(train_files, columns = ['filename'])
test_df = pd.DataFrame(test_files, columns = ['filename'])

In [5]:
# label 'em
train_df['hotdog'] = train_df['filename'].map(lambda x:
    'hotdog' if x.startswith('hotdog') else 'nothotdog')
test_df['hotdog'] = test_df['filename'].map(lambda x:
    'hotdog' if x.startswith('hotdog') else 'nothotdog')

In [6]:
# set up imagedatagenerators
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_dataframe(
        dataframe = train_df,
        directory = 'images/train',
        x_col = 'filename',
        y_col = 'hotdog',
        target_size=(300, 300),
        batch_size=32,
        class_mode='binary')
validation_generator = test_datagen.flow_from_dataframe(
        dataframe = test_df,
        directory = 'images/test',
        x_col = 'filename',
        y_col = 'hotdog',
        target_size=(300, 300),
        batch_size=32,
        class_mode='binary')

Found 3000 validated image filenames belonging to 2 classes.
Found 644 validated image filenames belonging to 2 classes.


In [7]:
# train baseline
train_df['hotdog'].value_counts(normalize = True)

hotdog       0.5
nothotdog    0.5
Name: hotdog, dtype: float64

In [8]:
# test baseline
test_df['hotdog'].value_counts(normalize = True)

hotdog       0.5
nothotdog    0.5
Name: hotdog, dtype: float64

In [9]:
# set up model
cnn_model = Sequential()

cnn_model.add(Conv2D( 
  filters = 6, # number of filters
  kernel_size = (3, 3), # height/width of filter
  activation = 'relu', # activation function 
  input_shape = (300, 300, 1) # shape of input (image)
))

cnn_model.add(MaxPooling2D(pool_size = (2, 2))) # dimensions of region of pooling

cnn_model.add(Conv2D(filters = 16,
                     kernel_size = (3, 3),
                     activation = 'relu'))

cnn_model.add(MaxPooling2D(pool_size = (2, 2)))

cnn_model.add(Flatten())

cnn_model.add(Dense(units = 128, activation = 'relu'))

cnn_model.add(Dense(units = 1, activation = 'sigmoid'))

cnn_model.compile(loss = 'binary_crossentropy',
                  optimizer ='adam',
                  metrics = ['accuracy'])


In [10]:
# fit model 1 - 5 epochs
cnn_model.fit(
        train_generator,
        epochs = 5,
        validation_data = validation_generator)

Epoch 1/5
94/94 [==============================] - 49s 489ms/step - loss: 0.7332 - accuracy: 0.5920 - val_loss: 0.6054 - val_accuracy: 0.6693
Epoch 2/5
94/94 [==============================] - 41s 439ms/step - loss: 0.6261 - accuracy: 0.6507 - val_loss: 0.5950 - val_accuracy: 0.6972
Epoch 3/5
94/94 [==============================] - 43s 453ms/step - loss: 0.6016 - accuracy: 0.6643 - val_loss: 0.6042 - val_accuracy: 0.6801
Epoch 4/5
94/94 [==============================] - 46s 485ms/step - loss: 0.5996 - accuracy: 0.6697 - val_loss: 0.5826 - val_accuracy: 0.6910
Epoch 5/5
94/94 [==============================] - 46s 487ms/step - loss: 0.5859 - accuracy: 0.6917 - val_loss: 0.6178 - val_accuracy: 0.6770


In [11]:
with open('./models/cnn_1.pkl', 'wb') as f:
    pickle.dump(cnn_model, f)

INFO:tensorflow:Assets written to: ram://0c51aca9-44c9-41c8-9b6f-7f5a1b5b40d5/assets


INFO:tensorflow:Assets written to: ram://0c51aca9-44c9-41c8-9b6f-7f5a1b5b40d5/assets


In [13]:
# fit model 1 - 10 epochs, batch size 128
cnn_model.fit(
        train_generator,
        epochs = 10, batch_size = 128,
        validation_data = validation_generator)

Epoch 1/10
94/94 [==============================] - 42s 442ms/step - loss: 0.4814 - accuracy: 0.7777 - val_loss: 0.6269 - val_accuracy: 0.7220
Epoch 2/10
94/94 [==============================] - 43s 451ms/step - loss: 0.4826 - accuracy: 0.7653 - val_loss: 0.7047 - val_accuracy: 0.6894
Epoch 3/10
94/94 [==============================] - 41s 435ms/step - loss: 0.4742 - accuracy: 0.7717 - val_loss: 0.6502 - val_accuracy: 0.7096
Epoch 4/10
94/94 [==============================] - 41s 430ms/step - loss: 0.4760 - accuracy: 0.7830 - val_loss: 0.5993 - val_accuracy: 0.7453
Epoch 5/10
94/94 [==============================] - 41s 431ms/step - loss: 0.4582 - accuracy: 0.7820 - val_loss: 0.6404 - val_accuracy: 0.7236
Epoch 6/10
94/94 [==============================] - 40s 429ms/step - loss: 0.4410 - accuracy: 0.7877 - val_loss: 0.6771 - val_accuracy: 0.7205
Epoch 7/10
94/94 [==============================] - 41s 431ms/step - loss: 0.4577 - accuracy: 0.7863 - val_loss: 0.6155 - val_accuracy: 0.7205

In [19]:
# set up model
cnn_model_2 = Sequential()

cnn_model_2.add(Conv2D( 
  filters = 6, # number of filters
  kernel_size = (3, 3), # height/width of filter
  activation = 'relu', # activation function 
  input_shape = (300, 300, 1) # shape of input (image)
))

cnn_model_2.add(MaxPooling2D(pool_size = (2, 2))) # dimensions of region of pooling

cnn_model_2.add(Conv2D(filters = 16,
                     kernel_size = (3, 3),
                     activation = 'relu'))

cnn_model_2.add(MaxPooling2D(pool_size = (2, 2)))

cnn_model_2.add(Flatten())

cnn_model_2.add(Dense(units = 128, activation = 'relu'))

cnn_model_2.add(Dense(units = 64, activation = 'relu'))

cnn_model_2.add(Dense(units = 1, activation = 'sigmoid'))

cnn_model_2.compile(loss = 'binary_crossentropy',
                  optimizer ='adam',
                  metrics = ['accuracy'])

In [20]:
# fit model 2 - 5 epochs, batch size 128
cnn_model_2.fit(
        train_generator,
        epochs = 10, batch_size = 128,
        validation_data = validation_generator)

Epoch 1/10
94/94 [==============================] - 62s 655ms/step - loss: 0.7140 - accuracy: 0.5560 - val_loss: 0.6325 - val_accuracy: 0.6320
Epoch 2/10
94/94 [==============================] - 61s 646ms/step - loss: 0.6574 - accuracy: 0.6147 - val_loss: 0.6206 - val_accuracy: 0.6599
Epoch 3/10
94/94 [==============================] - 62s 649ms/step - loss: 0.6313 - accuracy: 0.6397 - val_loss: 0.6604 - val_accuracy: 0.6149
Epoch 4/10
94/94 [==============================] - 63s 665ms/step - loss: 0.6049 - accuracy: 0.6697 - val_loss: 0.6127 - val_accuracy: 0.6755
Epoch 5/10
94/94 [==============================] - 63s 668ms/step - loss: 0.5844 - accuracy: 0.6847 - val_loss: 0.6356 - val_accuracy: 0.6382
Epoch 6/10
94/94 [==============================] - 62s 650ms/step - loss: 0.5737 - accuracy: 0.6943 - val_loss: 0.6580 - val_accuracy: 0.6739
Epoch 7/10
94/94 [==============================] - 61s 641ms/step - loss: 0.5616 - accuracy: 0.7117 - val_loss: 0.6190 - val_accuracy: 0.6724

In [26]:
# same as first one except early stopping

es = EarlyStopping(patience = 5)

# set up model
cnn_model_3 = Sequential()

cnn_model_3.add(Conv2D( 
  filters = 6, # number of filters
  kernel_size = (3, 3), # height/width of filter
  activation = 'relu', # activation function 
  input_shape = (300, 300, 1) # shape of input (image)
))

cnn_model_3.add(MaxPooling2D(pool_size = (2, 2))) # dimensions of region of pooling

cnn_model_3.add(Conv2D(filters = 16,
                     kernel_size = (3, 3),
                     activation = 'relu'))

cnn_model_3.add(MaxPooling2D(pool_size = (2, 2)))

cnn_model_3.add(Flatten())

cnn_model_3.add(Dense(units = 128, activation = 'relu'))

cnn_model_3.add(Dense(units = 1, activation = 'sigmoid'))

cnn_model_3.compile(loss = 'binary_crossentropy',
                  optimizer ='adam',
                  metrics = ['accuracy'])


In [27]:
# fit model 1 w/es - up to  50 epochs, batch size 64
cnn_model_3.fit(
        train_generator,
        epochs = 50, batch_size = 128,
        validation_data = validation_generator, callbacks = [es])

Epoch 1/50
94/94 [==============================] - 44s 460ms/step - loss: 1.1377 - accuracy: 0.5367 - val_loss: 0.6376 - val_accuracy: 0.6398
Epoch 2/50
94/94 [==============================] - 41s 429ms/step - loss: 0.6439 - accuracy: 0.6290 - val_loss: 0.6030 - val_accuracy: 0.6832
Epoch 3/50
94/94 [==============================] - 41s 433ms/step - loss: 0.6205 - accuracy: 0.6473 - val_loss: 0.6083 - val_accuracy: 0.6724
Epoch 4/50
94/94 [==============================] - 43s 455ms/step - loss: 0.6171 - accuracy: 0.6557 - val_loss: 0.6039 - val_accuracy: 0.6972
Epoch 5/50
94/94 [==============================] - 43s 458ms/step - loss: 0.5978 - accuracy: 0.6800 - val_loss: 0.5978 - val_accuracy: 0.7081
Epoch 6/50
94/94 [==============================] - 44s 463ms/step - loss: 0.5703 - accuracy: 0.6923 - val_loss: 0.5887 - val_accuracy: 0.6957
Epoch 7/50
94/94 [==============================] - 41s 436ms/step - loss: 0.5541 - accuracy: 0.7103 - val_loss: 0.5852 - val_accuracy: 0.7065

In [28]:
with open('./models/cnn_3.pkl', 'wb') as f:
    pickle.dump(cnn_model_3, f)

INFO:tensorflow:Assets written to: ram://ed815f83-f8c3-4675-85a4-8b1e78bde132/assets


INFO:tensorflow:Assets written to: ram://ed815f83-f8c3-4675-85a4-8b1e78bde132/assets


In [29]:
cnn_model_3.save('./models/cnn_3.ksm')

INFO:tensorflow:Assets written to: ./models/cnn_3.ksm\assets


INFO:tensorflow:Assets written to: ./models/cnn_3.ksm\assets
